In [4]:
import pandas as pd
df=pd.read_json("processed_crypto.json")
df_litecoin=df[df['asset_name']=='Litecoin']


In [5]:
df_date_created=df_litecoin.sort_values('created_utc')
df_date_created['created_utc']=df_date_created['created_utc'].str.slice(0,10)
df_date_created['sma20']=df_date_created['open'].rolling(window=20).mean()
df_date_created['high-low']=df_date_created['sma20'].shift(-1)-df_date_created['sma20']


In [6]:
df_date_created['buy-sell'] = df_date_created['high-low'].apply(lambda x: 0 if x <=0 else 1)

In [7]:
train_x=df_date_created[ (df_date_created['created_utc'].str.slice(0,4)== '2018')]
train_y=train_x['buy-sell']
train_x=train_x.drop(columns=['fin_subheading','news_title','news_url','reddit_title','symbol','tweet_text','tweet_text','asset_name','created_utc','reddit_domain','tweet_hashtags','fin_title','buy-sell',])


In [8]:
test_x=df_date_created[ (df_date_created['created_utc'].str.slice(0,4)== '2019') ]
test_y=test_x['buy-sell']
test_x=test_x.drop(columns=['fin_subheading','news_title','news_url','reddit_title','symbol','tweet_text','tweet_text','asset_name','created_utc','reddit_domain','tweet_hashtags','fin_title','buy-sell'])


In [9]:

train_x=train_x.fillna(train_x.mean())

test_x=test_x.fillna(test_x.mean())


In [10]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
x_scaler.fit(train_x)
X_train=x_scaler.transform(train_x)


/Users/mananparasher/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/mananparasher/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [27]:
lookback= 100

X=[]
for i in range(len(X_train)-lookback-1):
    t=[]
    for j in range(0,lookback):
        
        t.append(X_train[[(i+j)], :])
    X.append(t)


In [36]:
# import numpy as np

X= np.array(X)

X = X.reshape(X.shape[0],lookback, 30)


In [12]:
import numpy as np
train_y=np.reshape(list(train_y),(8252,-1))

In [13]:
from sklearn.preprocessing import StandardScaler
y_scaler = StandardScaler()
y_scaler.fit(train_y)
Y_train=y_scaler.transform(train_y)
#print(Y_train)

/Users/mananparasher/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/mananparasher/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [23]:
lookback= 100

P=[]
for i in range(len(Y_train)-lookback-1):
    t=[]
    for j in range(0,lookback):
        
        t.append(Y_train[[(i+j)], :])
    P.append(t)

In [34]:
#Y
#P= np.array(P)

#P = np.reshape(P.shape[0],100, 30)

#P=P.reshape((8151,100,-1))

#print(P.shape[0])
P

array([[[ 1.10863221],
        [ 1.10863221],
        [ 1.10863221],
        ...,
        [-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ]],

       [[ 1.10863221],
        [ 1.10863221],
        [ 1.10863221],
        ...,
        [-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ]],

       [[ 1.10863221],
        [ 1.10863221],
        [ 1.10863221],
        ...,
        [-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ]],

       ...,

       [[-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ],
        ...,
        [-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ]],

       [[-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ],
        ...,
        [-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ]],

       [[-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ],
        ...,
        [-0.9020124 ],
        [-0.9020124 ],
        [-0.9020124 ]]])

In [35]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [37]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1], 30)))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X, P, epochs = 10, batch_size = 32)



Epoch 1/10
8151/8151 [==============================] - 60s 7ms/step - loss: 0.4672
Epoch 2/10
8151/8151 [==============================] - 57s 7ms/step - loss: 0.2634
Epoch 3/10
8151/8151 [==============================] - 56s 7ms/step - loss: 0.2011
Epoch 4/10
8151/8151 [==============================] - 56s 7ms/step - loss: 0.1609
Epoch 5/10
8151/8151 [==============================] - 57s 7ms/step - loss: 0.1295
Epoch 6/10
8151/8151 [==============================] - 56s 7ms/step - loss: 0.1030
Epoch 7/10
8151/8151 [==============================] - 56s 7ms/step - loss: 0.0807
Epoch 8/10
8151/8151 [==============================] - 58s 7ms/step - loss: 0.0649
Epoch 9/10
8151/8151 [==============================] - 59s 7ms/step - loss: 0.0555
Epoch 10/10
8151/8151 [==============================] - 59s 7ms/step - loss: 0.0488


In [38]:
X_test=x_scaler.transform(test_x)

/Users/mananparasher/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [40]:
lookback= 100

X_t=[]
for i in range(len(X_test)-lookback-1):
    t=[]
    for j in range(0,lookback):
        
        t.append(X_test[[(i+j)], :])
    X_t.append(t)

In [41]:
X_t= np.array(X_t)

X_t = X_t.reshape(X_t.shape[0],lookback, 30)

In [43]:
predict=regressor.predict(X_t)

In [44]:
predict

array([[[-0.3165479 ],
        [-0.5993393 ],
        [-0.8515247 ],
        ...,
        [ 1.0727148 ],
        [ 0.97724515],
        [ 0.6436561 ]],

       [[-0.39922106],
        [-0.8410439 ],
        [-0.96485126],
        ...,
        [ 0.97719854],
        [ 0.6435336 ],
        [ 1.2416967 ]],

       [[-0.4585289 ],
        [-0.87377673],
        [-0.68136686],
        ...,
        [ 0.64345753],
        [ 1.2416888 ],
        [ 0.78712386]],

       ...,

       [[ 0.6646121 ],
        [ 0.43546742],
        [ 0.29105976],
        ...,
        [ 0.4001961 ],
        [ 0.43204466],
        [-0.01189166]],

       [[ 0.26583174],
        [ 0.14321303],
        [ 0.1844428 ],
        ...,
        [ 0.43204919],
        [-0.0118862 ],
        [-0.13347292]],

       [[ 0.10331939],
        [ 0.14082636],
        [ 0.53782827],
        ...,
        [-0.01188823],
        [-0.13347466],
        [-0.5513614 ]]], dtype=float32)

In [45]:
 b = predict.ravel()

109800


In [51]:
#= np.array(P)
predicted_stock_price = y_scaler.inverse_transform(predict)

#P=P.reshape((8151,100,-1))


In [53]:
B= b.flatten()
print(len(B))

109800
